In [1]:
import glob
import pandas as pd
import tqdm

Aggregate

In [2]:
paths12 = glob.glob('../data/chordata2/output/*/*/tables/ms12syn.tsv')
paths192 = glob.glob('../data/chordata2/output/*/*/tables/ms192syn.tsv')
paths12_internal = glob.glob('../data/chordata2/output/*/*/tables/ms12syn_internal.tsv')
paths192_internal = glob.glob('../data/chordata2/output/*/*/tables/ms192syn_internal.tsv')

In [19]:
total_data = []
for lbl, paths in zip(['12', '192', '12_internal', '192_internal'],
                      [paths12, paths192, paths12_internal, paths192_internal]):
    data = []
    for p in tqdm.tqdm(paths, 'Species-Gene'):
        gene, species = p.split('/')[5].split('__')
        spectrum = pd.read_csv(p, sep='\t')
        spectrum = spectrum.assign(Species=species, Gene=gene)
        data.append(spectrum)
    df = pd.concat(data)
    total_data.append(df.copy())
    df.to_csv(f'../data/chordata2/raw_spectum{lbl}.csv')

Species-Gene: 100%|██████████| 5925/5925 [00:11<00:00, 538.01it/s]


Process

In [8]:
total_data = [
    pd.read_csv('../data/chordata2/raw_spectum12.csv', index_col=0),
    pd.read_csv('../data/chordata2/raw_spectum192.csv', index_col=0),
    pd.read_csv('../data/chordata2/raw_spectum12_internal.csv', index_col=0),
    pd.read_csv('../data/chordata2/raw_spectum192_internal.csv', index_col=0),
]

In [9]:
total_data[1][total_data[1].ObsNum > 1]

,Mut,ObsNum,ExpNum,MutSpec,Species,Gene
2,A[T>C]A,1.99157,1.83333,0.039416,Eleotris_fusca,ND1
6,C[A>G]G,1.99216,3.99935,0.018074,Eleotris_fusca,ND1
17,G[G>A]T,1.99808,2.99984,0.024167,Eleotris_fusca,ND1
19,T[A>G]A,1.98928,1.99911,0.036106,Eleotris_fusca,ND1
20,T[A>G]C,1.99501,2.91658,0.024819,Eleotris_fusca,ND1
...,...,...,...,...,...,...
0,A[A>G]A,2.00000,1.90000,0.049075,Branchiostoma_belcheri,ND4L
1,A[C>T]G,2.00000,0.90000,0.103602,Branchiostoma_belcheri,ND4L
12,G[G>A]T,1.99993,1.19999,0.077700,Branchiostoma_belcheri,ND4L
17,T[A>G]G,3.99995,6.80000,0.027424,Branchiostoma_belcheri,ND4L


In [ ]:
df_info = pd.read_csv('../data/chordata2/info.csv')
df_info

,acc,gene,taxa,species
0,AB478558.1.8820.9502_BAH86124.1,A6,Chordata_7711;Leptocardii_2682552;Amphioxiform...,Branchiostoma belcheri
1,AB478590.1.8806.9488_BAH86540.1,A6,Chordata_7711;Leptocardii_2682552;Amphioxiform...,Branchiostoma floridae
2,Y09525.1.238.>921_CAA70712.1,A6,Chordata_7711;Leptocardii_2682552;Amphioxiform...,Branchiostoma lanceolatum
3,KP722183.1.159.>838_AJV90905.1,A6,Chordata_7711;Hyperoartia_117569;Petromyzontif...,Lampetra fluviatilis
4,FR669668.2.1432.>2101_CBW30998.1,A6,Chordata_7711;Hyperoartia_117569;Petromyzontif...,Lampetra planeri
...,...,...,...,...
18932,KY676708.1.1.519_AVM18678.1,ND6,Chordata_7711;Amphibia_8292;Caudata_8293;Pleth...,Eurycea braggi
18933,KY676695.1.1.519_AVM18665.1,ND6,Chordata_7711;Amphibia_8292;Caudata_8293;Pleth...,Eurycea spelaea
18934,EU880310.1.14048.13530_ACJ43568.1,ND6,Chordata_7711;Amphibia_8292;Caudata_8293;Salam...,Cynops ensicauda
18935,EU880313.1.14046.13528_ACJ43607.1,ND6,Chordata_7711;Amphibia_8292;Caudata_8293;Salam...,Cynops pyrrhogaster


In [21]:
total_data[1].groupby(['Species', 'Gene']).Mut.count()

Species                  Gene
Abavorana_luctuosa       ND1     192
Abbottina_rivularis      Cytb    192
Ablepharus_budaki        Cytb    192
Ablepharus_chernovi      Cytb    192
Ablepharus_kitaibelii    Cytb    192
                                ... 
Zosterops_poliogastrus   ND3     192
Zosterops_senegalensis   Cytb    192
                         ND3     192
Zosterops_simplex        ND2     192
Zygodontomys_brevicauda  CO1     192
Name: Mut, Length: 7478, dtype: int64

In [27]:
total_data[1]

,Mut,ObsNum,ExpNum,MutSpec,Species,Gene
0,A[A>G]C,1.00000,1.000000,0.036284,Eleotris_fusca,ND1
1,A[A>G]G,0.99709,0.999757,0.036187,Eleotris_fusca,ND1
2,A[T>C]A,1.99157,1.833330,0.039416,Eleotris_fusca,ND1
3,A[T>C]G,1.00000,2.000000,0.018142,Eleotris_fusca,ND1
4,C[A>G]A,0.99995,0.999992,0.036282,Eleotris_fusca,ND1
...,...,...,...,...,...,...
187,C[A>C]A,0.00000,1.000000,0.000000,Branchiostoma_belcheri,ND4L
188,G[C>T]A,0.00000,0.000000,0.000000,Branchiostoma_belcheri,ND4L
189,G[T>C]A,0.00000,0.000000,0.000000,Branchiostoma_belcheri,ND4L
190,A[A>T]T,0.00000,0.000000,0.000000,Branchiostoma_belcheri,ND4L


In [30]:
total_data[1][total_data[1].ObsNum >= 0.9]

,Mut,ObsNum,ExpNum,MutSpec,Species,Gene
0,A[A>G]C,1.00000,1.000000,0.036284,Eleotris_fusca,ND1
1,A[A>G]G,0.99709,0.999757,0.036187,Eleotris_fusca,ND1
2,A[T>C]A,1.99157,1.833330,0.039416,Eleotris_fusca,ND1
3,A[T>C]G,1.00000,2.000000,0.018142,Eleotris_fusca,ND1
4,C[A>G]A,0.99995,0.999992,0.036282,Eleotris_fusca,ND1
...,...,...,...,...,...,...
21,T[G>A]G,0.99999,2.399990,0.019425,Branchiostoma_belcheri,ND4L
22,T[G>A]T,1.00000,2.200000,0.021191,Branchiostoma_belcheri,ND4L
23,T[T>C]T,0.99999,12.400000,0.003760,Branchiostoma_belcheri,ND4L
24,T[T>G]G,0.99998,0.999999,0.046620,Branchiostoma_belcheri,ND4L


In [25]:
total_data[0][total_data[0].ObsNum > 1]

,Mut,ObsNum,ExpNum,MutSpec,Species,Gene
0,A>G,11.97280,30.91470,0.370075,Eleotris_fusca,ND1
1,C>T,4.98631,46.08120,0.103398,Eleotris_fusca,ND1
2,G>A,6.98846,21.08200,0.316758,Eleotris_fusca,ND1
3,T>C,8.98184,40.91500,0.209769,Eleotris_fusca,ND1
1,A>G,24.66940,37.15380,0.192592,Lasiurus_blossevillii,ND1
...,...,...,...,...,...,...
1,A>T,2.99996,12.90000,0.083793,Branchiostoma_belcheri,ND4L
3,C>T,3.99998,7.59997,0.189638,Branchiostoma_belcheri,ND4L
4,G>A,5.99982,11.29990,0.191311,Branchiostoma_belcheri,ND4L
7,T>C,2.99998,40.79990,0.026493,Branchiostoma_belcheri,ND4L


In [31]:
[x.split('/')[5] for x in paths12]

['ND1__Eleotris_fusca',
 'ND1__Lasiurus_blossevillii',
 'ND1__Anaxyrus_fowleri',
 'ND1__Lichanura_trivirgata',
 'ND1__Anaxyrus_woodhousii',
 'ND1__Plestiodon_lynxe',
 'ND1__Corvus_corax',
 'ND1__Hylomyscus_aeta',
 'ND1__Plecotus_macrobullaris',
 'ND1__Lasiopodomys_gregalis',
 'ND1__Coryphaena_hippurus',
 'ND1__Ovis_aries',
 'ND1__Cinclosoma_clarum',
 'ND1__Empidonax_alnorum',
 'ND1__Lasiurus_borealis',
 'ND1__Urosaurus_nigricaudus',
 'ND1__Xerotyphlops_vermicularis',
 'ND1__Polypedates_megacephalus',
 'ND1__Empidonax_flaviventris',
 'ND1__Macaca_mulatta',
 'ND1__Anguilla_rostrata',
 'ND1__Coregonus_sardinella',
 'ND1__Dremomys_rufigenis',
 'ND1__Alosa_alosa',
 'ND1__Natrix_tessellata',
 'ND1__Leptodactylus_fuscus',
 'ND1__Lepus_europaeus',
 'ND1__Guerlinguetus_aestuans',
 'ND1__Coregonus_pidschian',
 'ND1__Myotis_brandtii',
 'ND1__Salmo_salar',
 'ND1__Ursus_americanus',
 'ND1__Rana_sierrae',
 'ND1__Ectopistes_migratorius',
 'ND1__Carassius_gibelio',
 'ND1__Etroplus_suratensis',
 'ND1__